In [304]:
from l1_m_tags import m_tags
import pandas as pd


df_tags_master =  pd.read_csv("param_labels.csv")
df_videos = pd.read_csv("/Users/jesustellez/Desktop/aiDynamics/API Data extraction/Data/videos_eeuu.csv")
df_videos["Date"] = df_videos["publishedAt"].str.split("T", expand=True)[0]

remove_values = ["breaking news", "live video", "toriginal", "real time coverage", "news", "washington post", "anational", "apolitics", "spolitics", "snational", 
"anational", "washington post video" ,"md va", "aworld", "sworld", "wapo video"]

aux = m_tags(df_videos, df_tags_master, remove_values)
tags_matched, tags_mising = aux.master_tags()

/Users/jesustellez/Desktop/aiDynamics/API Data extraction/l1_youtubemodel/l1_m_tags.py:98: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jesustellez/Desktop/aiDynamics/API Data extraction/l1_youtubemodel/l1_m_tags.py:99: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jesustellez/Desktop/aiDynamics/API Data extraction/l1_youtubemodel/l1_m_tags.py:101: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/jesu

In [305]:
tags_matched.to_csv("/Users/jesustellez/Desktop/aiDynamics/API Data extraction/Data/l1/m_tags.csv")

In [320]:


tags = tags_mising.tags_clean.str.split(" ")
tags = tags.explode()

In [317]:
tags = tags.value_counts()

In [323]:
import nltk
import re

def clean_tags(text, flg_stemm = False, flg_lemm=True, lst_stopwords = None):
    ## Clean (convert to lowercase and remove punctuation and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower())

    ## Tolenize (Convert from string to List)
    lst_text = text.split()

    ## Remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (to remove -ly, -ing, etc.)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (Convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## Back to string from list
    text = " ".join(lst_text)
    return text

df_videos = pd.read_csv("/Users/jesustellez/Desktop/aiDynamics/API Data extraction/Data/videos_eeuu.csv")

df_videos["tags"] = df_videos.tags.str.split(",")
df_videos = df_videos.explode("tags")
lst_stopwords = nltk.corpus.stopwords.words("english")

df_videos["tags_clean"] = df_videos["tags"].apply(lambda x: clean_tags(
    x,
    flg_stemm=False,
    flg_lemm=True,
    lst_stopwords=lst_stopwords
))

In [326]:
analysis = df_videos[["video_id","tags_clean"]].merge(tags_matched[["tags_clean", "categorie"]], on="tags_clean", how="left")

In [325]:
tags_matched

,tags_clean,view_sum,like_sum,comment_sum,key_word,categorie
326,biden ukraine,257550,6550,1200.0,ukraine,war
327,biden ukraine announcement,1243300,9600,42700.0,ukraine,war
579,clean rave ukraine,354300,11750,3600.0,ukraine,war
581,cleanup rave ukraine,354300,11750,3600.0,ukraine,war
758,dj ukraine,354300,11750,3600.0,ukraine,war
...,...,...,...,...,...,...
589,coal,4094700,61500,0.0,coal,energy
2218,renewable energy,2246000,28800,57500.0,renewable,energy
689,damaging wind,213950,2000,200.0,wind,energy
1198,house windsor,12674500,93550,0.0,wind,energy


In [329]:
analysis = analysis[["video_id", "categorie"]].drop_duplicates()

In [333]:
analysis.groupby("categorie").agg({"video_id":"count"}).unstack()

          categorie  
video_id  energy          36
          financial       10
          health         165
          politics       503
          sport           23
          technologie     21
          terrorism       37
          war            297
dtype: int64

In [331]:
analysis

,video_id,categorie
0,w-NueyHZi0M,NaN
24,w-NueyHZi0M,politics
28,Q8dH8f6svEA,NaN
30,Q8dH8f6svEA,politics
40,mXXsWBvEJLQ,NaN
...,...,...
710979,UTiv7t436Vw,politics
711807,0UlurIUYS7o,NaN
712619,GusQdzkg1sI,NaN
712625,GusQdzkg1sI,health


In [334]:
analysis.video_id.drop_duplicates()

0         w-NueyHZi0M
28        Q8dH8f6svEA
40        mXXsWBvEJLQ
48        OTRYIcD4738
56        ktClzhOodkA
             ...     
710153    eudUmn7ANR8
710969    UTiv7t436Vw
711807    0UlurIUYS7o
712619    GusQdzkg1sI
713463    ogvnEoRbJro
Name: video_id, Length: 832, dtype: object